In [1]:
from models import GAN, SL2Z_DataModule
from pytorch_lightning import Trainer
import matrix_methods as mm

In [2]:
model = GAN()
sl2z_dm = SL2Z_DataModule('data.npy', batch_size=8192)

In [3]:
trainer = Trainer(
    max_epochs = 20,
    accelerator = 'auto',
    logger = False
    )

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [4]:
trainer.fit(model, sl2z_dm)

/home/gyueunlee/mambaforge/envs/pytorch-1.13.1/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:613: UserWarning: Checkpoint directory /home/gyueunlee/repos/sl2z-gan/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type          | Params
------------------------------------------------
0 | generator     | Generator     | 1.3 K 
1 | discriminator | Discriminator | 1.2 K 
------------------------------------------------
2.6 K     Trainable params
0         Non-trainable params
2.6 K     Total params
0.010     Total estimated model params size (MB)


Creating train dataloader


/home/gyueunlee/mambaforge/envs/pytorch-1.13.1/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=20` reached.


In [5]:
samples = model.generate_samples(20)

In [6]:
samples

tensor([[[ -37234.9609,   -1984.6097],
         [ -55542.1680,   29561.5605]],

        [[ -45925.2617,  -43825.6406],
         [ -38543.0391,  -18600.9375]],

        [[  15490.8223,    4311.6016],
         [   9169.2910,  -44036.7891]],

        [[ -22852.4316,  -15791.5898],
         [  42759.5547,  -40977.4375]],

        [[  24597.9785,  -54056.7500],
         [  97373.8750,  -96227.8594]],

        [[  10578.8428,  -91593.8984],
         [  29055.7500, -101477.6484]],

        [[ -15816.6865,  -29806.1699],
         [  73232.3672,  -70501.3438]],

        [[ -12647.3164,  -63940.5273],
         [  28389.4141,  -61052.3047]],

        [[   1630.5409,   -5894.9092],
         [   2283.1511,   -4180.3979]],

        [[  -9952.7451,    7554.3618],
         [  -2395.5754,  -33431.7695]],

        [[ -23120.0879,  -37079.1992],
         [  22058.8887,   17679.9160]],

        [[   -524.1798,  -20711.8242],
         [   5971.1138,   -7777.0259]],

        [[  54863.9414,  -79544.6250],
 

In [8]:
mm.det_multi(samples.detach().numpy())

array([-1.2109531e+09, -8.3492045e+08, -7.2170042e+08,  1.6116754e+09,
        2.8967045e+09,  1.5878134e+09,  3.2978739e+09,  2.5873818e+09,
        6.6426590e+06,  3.5083494e+08,  4.0916474e+08,  1.2774922e+08,
        1.1408589e+08, -3.2978811e+09,  1.7478237e+09,  3.5175905e+09,
        3.7546706e+09,  7.3000902e+08,  5.0990991e+09, -5.7187471e+09],
      dtype=float32)